## A Demonstration of the Elliptic Curve (EC) Cryptosystem

The EC cryptosystem demonstrated below combines Diffie&ndash;Hellman style symmetric key agreement with a digital signature to provide confidentiality, integrity and authentication in a message&ndash;exchange between two parties (Alice and Bob) over an insecure channel. A third party (Eve) eavesdropping on this channel has no practical way of learning the content of the message, nor any way of corrupting it without the receiving party knowing the message has been corrupted.

The key&ndash;agreement component of this cryptosystem provides for message confidentiality, and is known by the initials *ECDH*, which is short for *Elliptic Curve Diffie&ndash;Hellman* (alternatively *ECDHE*, the last *E* standing for *Ephemeral*, to indicate that the session key negotiated by the message&ndash;exchanging parties is to be used only once and then discarded). The digital signature component of this cryptosystem provides for message integrity and authenticity, and is known by the initials *ECDSA*, for *Elliptic Curve Digital Signature Algorithm*.

Elliptic curve cryptography represents a novel approach to the *discrete logarithm problem* (DLP) introduced by Diffie and Hellman in their 1976 paper [*New Directions in Cryptography*](https://ee.stanford.edu/~hellman/publications/24.pdf). The Diffie&ndash;Hellman protocol remains to this day the basis for key agreement over insecure channels. Elliptic&ndash;curve DLP has the advantage of requiring much shorter key lengths than traditional DH, however, to achieve equivalent security levels. This makes EC the only practical choice in size&ndash; and/or power&ndash;constrained applications, such as contactless smartcards and smart phones. Whereas an EC key of 256 bits in length provides 128&ndash;bit security (today considered the minimum acceptible security level), DH and RSA keys must be at least 3,072 bits long to provide the same level of security.

### The Source Code
The following sequence of function calls to the `ec` module (located in the [/src](https://github.com/dchampion/crypto/tree/master/code/src) directory of this repository) illustrates a session in which Alice signs and encrypts a private message, and then transmits the message, together with the signature, to Bob over an insecure channel. Meanwhile, an adversary&mdash;Eve&mdash;intercepts the message&ndash;signature pair transmitted on the insecure channel.

It is assumed that Alice, Bob and Eve all possess their own copy of the `ec` module; but that only Alice and Bob possess the signing and encryption keys generated by this module to provide the aforementioned protections.

Values below surrounded in square brackets [ ] are public (i.e., they can be transmitted over the insecure channel with no compromise to the security of the system, even if they are observed by Eve), and those that are not in square brackets are private (i.e., they must not be transmitted over the insecure channel).

Explanations of behavior appear *below* the code snippets and their outputs.

In [1]:
import sys
sys.path.append("../code/src")
import ec
sys.path.append("../code/test")
import sym

Import the `ec` module, and a module `sym` that implements an elementary symmetric cipher that Alice and Bob will use in the protocol.

In [2]:
ec.new_curve(17,2,2,5,1,19,1,5)

The `ec` module defines a NIST&ndash;/SECG&ndash;sanctioned ellipctic curve by default; specifically, the [*secp256k1*](https://www.secg.org/sec2-v2.pdf#subsubsection.2.4.1) curve (we assume this curve is secure because it has secured the Bitcoin blockchain since 2009). For the purposes of this demonstration, however, Alice will define a much smaller (and therefore much less secure) curve.

An elliptic curve is defined by the congruence relation *y<sup>2</sup> &Congruent; x<sup>3</sup> + ax + b (mod p)*, where *x* and *y* are coordinates on a cartesian plane, *a* and *b* are constant coefficients, and *p* is a prime modulus that confines results to a finite integral field.

More formally, to define the curve above, we supplied the following domain parameters:

* A field parameter *p* (17) (this is also the prime modulus)
* A coefficient *a* (2)
* A coefficient *b* (2)
* An *x*&ndash;coordinate for the base point *G* (5)
* A *y*&ndash;coordinate for the base point *G* (1)
* The order *n* of the base point *G* (19)
* The cofactor *h* of the curve point group, which is the total number of points on the curve divided by the order of *G*, or *n* (1)
* An optional parameter that specifies the number of iterations to test the curve's strength against a certain class of attacks (5) (5 is the value supplied here because the default&mdash;100&mdash;will cause this small curve to fail validation).

If any of these parameters is invalid, `ec.new_curve` will raise an exception. As it turns out, a curve with these parameters is valid (albeit wholly insecure due to its small size).

In [3]:
print(ec._p, ec._a, ec._b, ec._Gx, ec._Gy, ec._n, ec._h)

17 2 2 5 1 19 1


Here, we print the aforementioned domain parameters to the console.

In [4]:
pt = ec._G
print(pt) # Print base point
while pt != ec._i:
    pt = ec.add(ec._G, pt)
    print(pt) # Print remaining points...

[5, 1]
[6, 3]
[10, 6]
[3, 1]
[9, 16]
[16, 13]
[0, 6]
[13, 7]
[7, 6]
[7, 11]
[13, 10]
[0, 11]
[16, 4]
[9, 1]
[3, 16]
[10, 11]
[6, 14]
[5, 16]
[None, None]


Here, we print all the points on the curve to the console. Note that the first point in the list, the base point `[5, 1]`, comes from the coordinates Alice specified in the 4<sup>th</sup> and 5<sup>th</sup> parameters to `ec.new_curve`. Also note that the last point is `[None, None]`, which in an elliptic&ndash;curve point group is the identity element (this is also referred to as the *point at infinity*).

The total number of points in the group is 19, which Alice specified in the 6<sup>th</sup> parameter to `ec.new_curve` (this can be verified by counting the coordinate pairs in the list above).

Note that this code is not required for the protocol; it is only included to provide a mental model of the elliptic curve.

In [5]:
dA, QA = ec.generate_keypair()

Here's where the protocol begins. Alice generates a pair of keys, and stores them in *dA* and *[QA]*; these are her private and public keys, respectively. The suffixed capital letter *A* denotes that these keys belong to Alice.

Alice transmits her public key *[QA]* to Bob, but she must keep her private key *dA* secret.

In [6]:
print(dA, QA)

10 [7, 11]


If we print Alice's keys, we see that her private key is an integer, and her public key a point on the curve. Note that her private key is the 1&ndash;based index of the curve in the list we printed above; that is, it is the *dA*<sup>th</sup> point in the list.

More formally, using elliptic curve math, the base point `[5, 1]` has been *added* to itself *dA* times to arrive at the point represented by the public key *[QA]*. *dA* is selected randomly from the range *1 < dA < n - 1*, where *n* is the number of points on the curve, to produce the public key.

In [7]:
dB, QB = ec.generate_keypair()

Now it's Bob's turn to generate a key pair (suffixed with the capital letter *B* this time to denote these keys belong to Bob). As with Alice's keys, Bob transmits his public key *[QB]* to Alice, and keeps his private key *dB* secret.

In [8]:
print(dB, QB)

2 [6, 3]


If we print Bob's keys, we see a familiar pattern.

In [9]:
kSessionA = ec.generate_session_key(dA, QB)

Now Alice computes a session key *kSessionA*, using her private key *dA* and Bob's public key *[QB]*.

If you are familiar with traditional Diffie&ndash;Hellman, you might recognize a pattern here. The essential property of DH is expressed in the signature of `ec.generate_session_key`; namely, a secret session key is computed using one party's private key in combination with the other party's public key (in the present case these are Alice's private key *dA* and Bob's public key *[QB]*).

Whereas in traditional DH the public key is raised to the power of the private key (both integers), in ECDH the private key (an integer) is *multiplied* by the public key (a curve point). The result of both operations is a session key; in the former this key is an integer, and in the latter it is a curve point (*multiplication* is just another name for *repeated addition*, and in fact *addition* is the only operator supported by elliptic&ndash;curve point groups).

One additional implementation detail is worth noting here: One might wonder how it is that a curve point `[x, y]`, generated by the multiplication of an integer to the base point, is represented as an numeric key in a symmetric cipher. The answer is that the x-coordinate of the public key&mdash;which itself is an integer in the range *0 < x < _p*, where *_p* is the prime modulus in the curve group&mdash;is extracted from the point, and just that is used as the session key.

In [10]:
kSessionB = ec.generate_session_key(dB, QA)

Now Bob computes a session key *kSessionB*, using his private key *dB* in combination with Alice's public key *[QA]*. If the essential property of DH holds, Alice's and Bob's session keys&mdash;*kSessionA* and *kSessionB*&mdash;should be identical.

Further, Eve, having only observed Alice's and Bob's public keys, cannot feasibly derive the session key with just that knowledge. The only way for Eve to find the session key is to add curve points repeatedly, starting with the base point, until she finds the  session key. With a standard EC key length of 256 bits, this would require ~2<sup>128</sup> trial point additions, a process that at current computer speeds would take many, many years (of course in the present example, using a curve with just 19 points, it would take Eve at most that many steps to find the session key).

In [11]:
print(kSessionA == kSessionB)

True


We prove here that *kSessionA* is in fact equivalent to *kSessionB*.

In [12]:
mA = "Sign and encrypt me!"

Now Alice composes a message for Bob, and stores it in the variable *mA*.

In [13]:
S = ec.sign(dA, mA)

Then Alice signs the message and stores the signature in *[S]*. To sign the message, Alice must supply her private key *dA* to the signing function `ec.sign`.

In [14]:
mAC = sym.encrypt(kSessionA, mA)

Next, Alice encrypts the message *mA*, supplying it and her session key *kSessionA* to the function `sym.encrypt`, and stores the ciphertext in *[mAC]*.

Alice is now ready to transmit the ciphertext *[mAC]*, along with the signature *[S]*, to Bob.

In [15]:
print(mAC)

108182406554699997314868250957460824610098768037654763450253334871593747383996


Here, we print *[mAC]*, revealing a ciphertext that is wholly indistinguishable from its plaintext *mA*.

In [16]:
mB = sym.decrypt(kSessionB, mAC)

Bob, having received the ciphertext and signature from Alice, first decrypts the ciphertext *[mAC]*, supplying it and his session key *kSessionB* to the function `sym.decrypt`.

Bob stores the result in the variable *mB*.

In [17]:
print(mB)

Sign and encrypt me!


Here we print *mB*, proving that it matches the plaintext *mA* Alice encrypted into *[mAC]* and transmitted to Bob. Note Alice transmitted the ciphertext *[mAC]* to Bob, ***not*** the plaintext *mA*.

Thus far Alice has transmitted her message *confidentially* to Bob; what remains is for Bob to verify the *integrity* and *authenticity* of the message.

In [18]:
print(ec.verify(QA, mB, S))

True


Here Bob verifies the *integrity* and *authenticity* of the message (the former property ensures the message has not been altered, and the latter that it was signed with Alice's private key). Bob performs this verification using the function `ec.verify`, supplying it Alice's public key *[QA]* and signature *[S]*, both of which were transmitted to him by Alice, and the plaintext of the message *mB* he decrypted in the previous step.

If the function `ec.verify` returns `True`, Bob can be satisfied that the conditions of confidentiality, integrity and authenticity have all been met, and the session is complete.

If the function returns `False`, Bob must reject the message and assume the system is being attacked.